In [ ]:
# %load 渐进式剪枝.py

import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn.utils.prune as prune
from vgg_quant import*
     
import torchvision
import torchvision.transforms as transforms

global best_prec
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu") 
    
batch_size = 128
model = VGG16_quant()


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
print_freq = 100
def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()
            output = model(input)
            loss = criterion(output, target)
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        


TARGET_SPARSITY = 0.8  
N_EPOCHS = 10
PRUNE_STEPS = 4  

if PRUNE_STEPS == 2:
    PRUNE_SCHEDULE = {
        5: 0.4,   
        10: 0.8, 
    }
    PATH_prune = "gradual_2step_0.8_ep10_vgg_pruned.pth"
elif PRUNE_STEPS == 4:
    PRUNE_SCHEDULE = {
        2: 0.2,  
        4: 0.4, 
        7: 0.6, 
        10: 0.8, 
    }
    PATH_prune = "gradual_4step_0.8_ep10_vgg_pruned.pth"
def apply_pruning_with_target_sparsity(model, target_sparsity):
    for name, module in model.named_modules():
        if isinstance(module, QuantConv2d):
            if hasattr(module, 'weight_orig'):
                prune.remove(module, 'weight')
            prune.l1_unstructured(module, name='weight', amount=target_sparsity)
    
    total_zeros = 0
    total_params = 0
    for module in model.modules():
        if isinstance(module, QuantConv2d):
            mask = module.weight_mask if hasattr(module, 'weight_mask') else (module.weight != 0)
            total_zeros += (mask == 0).sum().item()
            total_params += mask.numel()
    
    actual_sparsity = total_zeros / total_params if total_params > 0 else 0

lr = 1e-1
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

load_best_valid_parameters = '4bit_VGG_best_valid_acc.pth'
checkpoint = torch.load(load_best_valid_parameters)
model.load_state_dict(checkpoint['state_dict'])

model.cuda()
model.train()
for epoch in range(N_EPOCHS):
    # monitor training loss
    train_loss = 0.0
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    train_loss = train_loss / len(trainloader.dataset)
    print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss:.6f}')
    
    current_epoch = epoch + 1
    if current_epoch in PRUNE_SCHEDULE:
        target_sparsity = PRUNE_SCHEDULE[current_epoch]
        apply_pruning_with_target_sparsity(model, target_sparsity)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': train_loss,
        'prune_schedule': PRUNE_SCHEDULE,
        'target_sparsity': TARGET_SPARSITY,
    }, PATH_prune)
    
    scheduler.step()

print("=" * 60)
print(f"渐进式剪枝训练完成! 最终稀疏度: {TARGET_SPARSITY*100:.0f}%")
print(f"模型已保存到: {PATH_prune}")
